# 데이터 제작

내가 관심 있는 블로그 주제를 선별한 후 요약하고 전달해주는 에이전트를 만들고 싶다면? 
에이전트 평가 및 최적화를 위한 실습 데이터를 제작해보자.


In [1]:
%pip install 'gql[requests]'


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Velog 블로그 제목 크롤링

개발 관련 블로그 플랫폼 Velog에서 블로그 제목을 크롤링한다.

In [2]:
from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport

transport = RequestsHTTPTransport(
    url='https://v3.velog.io/graphql',
    headers={
		"Origin": "https://velog.io",
	},
    use_json=True,
)

In [3]:
client = Client(transport=transport, fetch_schema_from_transport=True)

In [12]:
query = gql('''
query trendingPosts($input: TrendingPostsInput!) {
  trendingPosts(input: $input) {
    title
    short_description
  }
}
''')

In [13]:
result = client.execute(query, variable_values={"input": {"limit": 100, "offset": 100, "timeframe": "week"}})

In [14]:
len(result['trendingPosts'])

100

In [15]:
result['trendingPosts'][0]

{'title': '[코드잇 스프린트 Node.js 백엔드 1기] 위클리 페이퍼 - 1주차',
 'short_description': 'Q1. URL의 구성 요소 중 쿼리가 무엇인지 실제 웹 서비스의 URL을 예시로 설명해 주세요.\n\n1. URL의 구성요소\n\n 먼저 네트워크상에서 자원(Resource)을 주고받으려면 자원을 식별할 수 있\n어야 합니다. 자원을 식별할 수 있는 정보를 URI(Unifor '}

Title과 Short Description을 보고 AI에 관련한 블로그 글인지 판별하는 에이전트를 만들어 봅시다.
이를 위해 csv 파일로 저장합니다. 

In [16]:
titles = []
short_descriptions = []

for post in result['trendingPosts']:
    titles.append(post['title'])
    short_descriptions.append(post['short_description'])
	
import pandas as pd
df = pd.DataFrame({'title': titles, 'short_description': short_descriptions})

In [17]:
df.head()

,title,short_description
0,[코드잇 스프린트 Node.js 백엔드 1기] 위클리 페이퍼 - 1주차,Q1. URL의 구성 요소 중 쿼리가 무엇인지 실제 웹 서비스의 URL을 예시로 설...
1,FSD 간단하게 시작해보기,공식홈피를 기반 했지만 제 입맛에 따라 해석 했음을 알려드립니다.\n\n프롤로그\n...
2,[면접을 위한 CS 전공지식 노트] 2.3 네트워크 기기,면접을 위한 CS 전공지식 노트 - 2.3 네트워크 기기
3,🎓 Reinforcement learning - 1,"🎓 강화학습(Reinforcement Learning, RL)이란?\n>강화학습(R..."
4,TIL-2,논리적으로 제품을 만들 수 있도록 도와주는 프레임워크 중 하나.사용자에 대한 이해를...


In [18]:
import os
import pathlib

root_dir = pathlib.PurePath(os.path.dirname(os.getcwd())).parent
data_dir = os.path.join(root_dir, 'data')

In [19]:
df.to_csv(os.path.join(data_dir, 'agent', 'original_data.csv'), index=False)

## 데이터 라벨링

어떻게 "AI 관련 포스트"인지 아닌지 평가할 수 있을까요?
배웠던 내용을 조금만 떠올려보면 쉽게 알 수 있습니다.

이제 평가를 위해 라벨링을 진행합니다.

[Label Studio](https://labelstud.io/guide/install.html)를 활용합시다.

In [ ]:
%pip install label-studio

In [ ]:
!label-studio

In [23]:
# Setup for the label studio
'''
<View>
<Header value="$title"></Header>
  <Text name="text" value="$short_description"></Text>
  <Choices name="sector" toName="text" choice="single" showInLine="true">
   <Choice value="AI"/>
   <Choice value="Frontend"/>
   <Choice value="Backend"/>
   <Choice value="CS" />
   <Choice value="DevOps" />
   <Choice value="etc" />
  </Choices>
</View>
'''

'\n<View>\n<Labels name="lbl" toName="title">\n    <Label value="AI"/>\n    <Label value="Front-end"/>\n  \t<Label value="Back-end"/>\n    <Label value="etc"/>\n  </Labels>\n  <Text name="title" value="$title"/>\n  <Text name="description" value="$short_description"/>\n</View>\n'